In [1]:
# https://quantumcomputing.stackexchange.com/questions/24050/how-to-implement-a-exponential-of-a-hamiltonian-but-non-unitary-matrix-in-qisk

# control gate: https://qiskit.org/documentation/stubs/qiskit.circuit.ControlledGate.html

# circuit https://www.nature.com/articles/s41598-022-17660-8
import numpy as np
import configparser
from sklearn.preprocessing import normalize
import collections
import json
from tqdm import trange
import scipy
from scipy import stats
from numpy.random import default_rng
import random

from qiskit.circuit import ControlledGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.extensions import UnitaryGate
from qiskit.tools.visualization import plot_histogram

from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.list_ops import SummedOp
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki, PauliSumOp

In [2]:
def get_gate(A, n):    
    pauli_op = PauliSumOp(SparsePauliOp.from_operator(A))
    phi = Parameter('ϕ')
    evolution_op = (phi * pauli_op).exp_i() # exp(-iϕA)
    trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evolution_op).bind_parameters({phi: np.pi/n})
    #----control---------
    gate = trotterized_op.to_circuit()
    gate.name = f"e^(i*A*pi/{n})"
    gate.label = f"e^(i*A*np.pi/{n})"
    gate = gate.to_gate().control()
    #---------------------
    return gate

In [3]:
def get_circuit(A, b, eigns_qub_num=2):    
    n_b = int(np.log2(len(b)))
    n_ = eigns_qub_num #3 is an optimal
    n_ancilla = 1
    n_cl = n_b#+1
    # quantum circuit initialization
    b_x = QuantumRegister(n_b, 'b and x')
    eigs = QuantumRegister(n_, 'eigenvalues')
    ancilla = QuantumRegister(n_ancilla, 'ancilla')
    classical = ClassicalRegister(n_cl, 'classical')
    qc = QuantumCircuit(b_x, eigs, ancilla, classical)
    # b-vector state preparation
    qc.initialize(b, b_x)
    qc.h(eigs)
    qc.x(ancilla)
    # qc.barrier()
    # Matrix exponentiation
    for i in range(0, len(eigs)):
        gate = get_gate(A, 2**(i+1))
        qc.append(gate,[eigs[i], *b_x])
    qc.barrier()
    # Phase estimation
    for j in range(len(eigs)-1, 0, -1):
        qc.h(eigs[j])
        for m in range(j):
            qc.crz(-np.pi/float(2**(j-m)), eigs[j], eigs[m])
    # qc.barrier()
    # As I understood, we wncode ancilla qubit to be sure that result will be correct
    for j in range(1, 1+n_):
        qc.cry(np.pi/(2**j), n_b+n_-j, n_b+n_)
    # qc.barrier()
    # Inverse quantum Fourier transform
    for j in range(len(eigs)-1, 0, -1):
        qc.h(eigs[j])
        for m in range(j):
            qc.crz(np.pi/float(2**(j-m)), eigs[j], eigs[m])
    # qc.barrier()
    # Eigenvalues storing in the vecor b register
    for i in range(len(eigs), 0, -1):
        gate = get_gate(A, -2**(i+1))
        qc.append(gate,[eigs[i-1], *b_x])
    # qc.barrier()
    # qubits measurement. I do not measure the ancilla qubit
    qc.measure(b_x, classical)
    return qc

In [4]:
def execute_circ(qc, shots = 2048):    
    simulator = Aer.get_backend('qasm_simulator')
    circ = transpile(qc, simulator)
    result = execute(qc, backend=simulator, shots=shots).result()
    return result, sum(circ.count_ops().values())

# Calculated result

In [5]:
def get_solution(result, shots = 2048):
    counts = result.get_counts()
    probabilities = counts.copy()
    probs_upd = {}
    for k, v in probabilities.items():
        value = v/shots
        probabilities[k] = value
    vect = np.array([v for k,v in probabilities.items()])
    vect.sort()
    return vect

# Real normalized result

In [6]:
def get_real_solution(x):
    real_x_norm = (np.array(x) / np.linalg.norm(x))**2
    real_x_norm.sort()
    return real_x_norm
# np.round(real_x_norm, 5)

# Error

In [7]:
def get_error(vect, real_x_norm):
    mse = (np.square(vect - real_x_norm)).mean()
    abs_err = np.mean([np.abs((v-r)/r) for v,r in zip(vect, real_x_norm)])
    return mse, abs_err

In [8]:
import scipy.linalg as la
def get_data():    
    fail = True
    while fail:
        try:
            size = random.choice([2, 4, 8, 16])
            des = np.random.uniform(low=0, high=1, size=size)
            n = len(des)
            s = np.diag(des)
            q, _ = la.qr(np.random.rand(n, n))
            A = q.T @ s @ q
            
            v = np.random.rand(size)
            b = v / np.linalg.norm(v)
            
            x = np.linalg.solve(A, b)
            fail = False
        except np.linalg.LinAlgError:
            fail = True
    return A, b, x

In [9]:
def save_data(example, A, b, x):
    config = configparser.ConfigParser()
    config[example] = {'A': A,
                       'b': b,
                       'x': x,
                      }
    with open("random_config.ini","a+") as configfile:
        config.write(configfile)

In [ ]:
with open('random_data.json', 'a+') as fp:
            json.dump({}, fp)
for example in range(201):
    error_data = {}
    error_data['example '+str(example)] = {}
    A, b, x = get_data()
    save_data(example, A, b, x)
    print(f"====== example {example} ========")
    for i in trange(2, 21):    
        circ = get_circuit(A, b, i)
        result, op_num = execute_circ(circ)
        x_prime_norm = get_solution(result)
        x_norm = get_real_solution(x)
        if len(np.zeros(len(x_norm) - len(x_prime_norm))) > 0:
            x_prime_norm = np.concatenate((x_prime_norm, np.zeros(len(x_norm) - len(x_prime_norm))))
        mse, abs_err = get_error(x_prime_norm, x_norm)
        # print(f"mse {np.round(mse, 4)}, abs {np.round(abs_err, 4)}, depth {circ.depth()}, number of operators {circ.num_qubits} "
        #       f"amount of operations is {op_num} for {i} eigenvalues qubits", '\n')
        error_data['example '+ str(example)][i] = {
            'mse': mse,
            'abs': abs_err,
            'depth': circ.depth(),
            'op_num': op_num,
             }
    # with open('random_data.json', 'a+') as fp:
    #     json.dump(error_data, fp)
        
    with open('random_data.json') as json_data_file:
        data = json.load(json_data_file)

    data.update(error_data)
    with open('random_data.json', 'w+') as fp:
        json.dump(data, fp)

====== example 0 ========


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:42<00:00,  2.22s/it]


====== example 1 ========


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [02:38<00:00,  8.32s/it]


====== example 2 ========


...

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [11:11<00:00, 35.37s/it]


====== example 57 ========


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:39<00:00,  2.09s/it]


====== example 58 ========


 84%|█████████████████████████████████████████████████████████████████████             | 16/19 [01:45<00:32, 10.80s/it]

In [87]:
# with open('random_data.json', 'w') as fp:
#     json.dump(error_data, fp)